In [35]:
import random
import copy

#Taking the input
input=['5','l 100','l 450','d 500','l 7923','d 9055']
#input= ['7', 'l 120','l 289','d 475','l 195','d 6482','l 160','d 935']

#leng= total tranzaction possible aka first digit of input line
leng=int(input[0])

#array= all tranzactions input EX- [l100 means -100, d500 means +500] -> [-100,-450,500,-7923,9055]
array=[]
for i in range(1,leng+1):
  if 'l' in input[i]:
    temp= input[i].split()
    array.append(-int(temp[1]))
  elif 'd' in input[i]:
    temp= input[i].split()
    array.append(int(temp[1]))
#print("INP",array)


#FUNCTION : to check if all values are 0 in an array EX- returnes true is all zero
def ALL_0_checker(arr):
  f=True
  for i in arr:
    if i==1:
      f=False
  return f

#populating the states by 0/1 randomizing EX(0 stating that dont pick the corresponding transaction and 1 as do pick)
popu=[]

#generating 10 possibilities EX- lenth of popu=10 popu=[[leng],[leng],...]
i=0

while i!=10:
  #t_arr= [0,1 combinations array--populations]
  t_arr=[]

  #each having 7 tranzactions to pick from
  for idx in range(leng):
    binary = random.randint(0, 1)
    t_arr.append(binary)

  #checking if any list in popu is all zeros
  f=ALL_0_checker(t_arr)

  if f==True:
    #ignore this one
    pass

  elif t_arr in popu:
    #generated same list - redundancy - ignore
    pass

  else:
    popu.append(t_arr)
    i+=1

#print("POPU",popu)


#FUNCTION: LOW TO HIGH PROBABILITY TAKER EX- Takes a  list, returnes the prob of each one of those
def Low_to_high_prob_maker(a):
  b=[]
  t=0
  for i in a:
    t+=i
  for j in a:
    b.append(j/t)
  return (b)


#FUNCTION: FITNESS probability calculation -> returns probability of each fitness
def FITNESS(popu,array):
  #Fitness solid value of each population
  fitness=[]

  #Taking each population combination in b_list and finding it's fitness - appending in fitness each
  for b_list in popu:

    t_sum=0
    for i in range(leng):
      x=b_list[i]
      y=array[i]
      #0*value=-(don't pick), 1*value=pick
      t_sum+=(x*y)

    fitness.append(abs(t_sum))

  for z in range(len(popu)):
    if fitness[z]==0:
      return [popu[z]]


  #Total sum of all fitness for Probability calc
  total = 0
  for ele in fitness:
      total += ele

  fit_prob=[]
  for i in fitness:
    low=(i)/total
    fit_prob.append(1/low)

  #Better parents hold lower probability so pass through the func
  return Low_to_high_prob_maker(fit_prob)


#FUNCTION: Cumalitive probability calculator -> EX [0,0.3,0.7,1]
def Cumalitive(list):
  new=[list[0]]
  for i in range (1,len(list)):
    s=new[i-1]
    s+=list[i]
    new.append(s)
  return new

#FUNCTION: Picks two for crossover randomly, using cumalitive probability, returns the picked indexes
def Pick_for_Crossover(cuma):
  r1=random.random()
  r2=random.random()

  idx_1= None
  for i in range(len(cuma)):
    if cuma[i]>=r1 :
      idx_1=i
      break
  idx_2= None
  for i in range(len(cuma)):
    if cuma[i]>=r2 :
      idx_2=i
      break

  return [idx_1,idx_2]


#Crossover offspring generator for two picked parents(2 lists in popu) -> returns two new offsprings in a list [[os1],[os2]]
def CROSSOVER(p_1,p_2,leng):

  while True:
    #randomly picks an idx to start crossover
    idx=random.randint(1, leng-2)
    #seperate the FIRST part before the idx
    c1=p_1[:idx]
    c2=p_2[:idx]
    #LAST part of parents after the idx
    p_1_2=p_1[idx:leng+1]
    p_2_2=p_2[idx:leng+1]
    #new lists of crossover
    c1.extend(p_2_2)
    c2.extend(p_1_2)
    #checking for all zeros
    check1=ALL_0_checker(c1)
    check2=ALL_0_checker(c2)

    if check1 ==True or  check2 ==True:
      pass
    else:
      return [c1,c2]
      break


#-----------WHILE LOOP STARTS----------------

j=0
result=-1
while j<=350:
  j+=1

  #1.CROSSOVER
  #i)prob_2 takes the fitness probability for each popu EX len(prob_2)=len(popu)=10
  prob_2=FITNESS(popu,array)

  if len(prob_2)==1:
    result=prob_2[0]
    break

  #ii)storing the cumallitive probability of prob_2
  cuma= Cumalitive(prob_2)
  #print("cumalitive",cuma)

  #iii) picked has two idx in a list picked for crossover
  picked= Pick_for_Crossover(cuma)


  #iv)crossed has two offspring generated after crossover
  crossed= CROSSOVER(popu[picked[0]],popu[picked[1]],leng)

  #Making a copy of crossed to check for all zeros list after mutation
  check_zero=copy.deepcopy(crossed)
  #print("CROSSED",crossed)

  #2MUTATION
  #mutating amoung only the two offsprings generated
  for i in range(2):
    #i)taking two probabilities for mutation
    setted=0.3
    r_m= random.random()

    if r_m<setted:
      #ii)taking a random idx and value
      idx_sel= random.randint(0, leng-1)
      new_val=random.randint(0, 1)
      #using copy of crossed, check if mutation makes it all zeros list
      check_zero[i][idx_sel]=new_val
      c_m=ALL_0_checker(check_zero[i])
      #print("checker",c_m)
      if c_m==True:
        pass
      else:
        crossed[i][idx_sel]=new_val

  #Some crossover may not be picked for mutation-> checking for all zeros and already existance in popu
  for x in range(2):
    c_z=ALL_0_checker(crossed[x])
    if c_z==True or crossed[x] in popu:
      pass
    else:
      #adding new parents to popu after crossover and mutation
      popu[picked[x]]=crossed[x]


if result==-1:
  print(result)
else:
  for item in result:
    print(item,end=" ")





-1
